In [2]:
import numpy as np
import gc;from datetime import datetime;import pandas as pd
import warnings;warnings.simplefilter(action='ignore', category=Warning)

https://www.kaggle.com/competitions/child-mind-institute-detect-sleep-states/data

In [ ]:
train_parquet_path = "train_series.parquet"
test_parquet_path = "test_series.parquet"
train_events_path = "train_events.csv"
train_series = pd.read_parquet(train_parquet_path)
test_series = pd.read_parquet(test_parquet_path)
train_events = pd.read_csv(train_events_path)

In [ ]:
series_has_NaN = train_events.groupby('series_id')['step'].apply(lambda x: x.isnull().any())
#series_has_NaN.value_counts()
no_NaN_series = series_has_NaN[~series_has_NaN].index.tolist()
 # incomplete events data
no_NaN_series.remove('31011ade7c0a')
no_NaN_series.remove('a596ad0b82aa') 

In [ ]:
def get_train_series(series):
    train_series = pd.read_parquet("train_series.parquet", filters=[('series_id','=',series)])
    train_events = pd.read_csv("train_events.csv").query('series_id == @series')
    train_events = train_events.dropna()
    train_events["step"]  = train_events["step"].astype("int")
    train_events["awake"] = train_events["event"].replace({"onset":1,"wakeup":0})
    train = pd.merge(train_series, train_events[['step','awake']], on='step', how='left')
    train["awake"] = train["awake"].bfill(axis ='rows')
    # final section:
    #train_events.groupby('series_id').tail(1)["event"].unique()
    # Result: the last event is always a "wakeup"
    train['awake'] = train['awake'].fillna(1) # awake
    train["awake"] = train["awake"].astype("int")
    return(train)
clean_train_data = []
for series_id in no_NaN_series:
    train = get_train_series(series_id)
    clean_train_data.append(train)
    del train
    gc.collect();

In [ ]:
Zzzs_train = pd.concat(clean_train_data).reset_index(drop=True)
print(Zzzs_train["series_id"].nunique(), "indivudual sleep training series")

In [3]:
file_path = 'C:/temp/clean_zzz.csv'
#Zzzs_train.to_csv(file_path, index=False)
zzz = pd.read_csv(file_path)

In [4]:
zzz.head()

,series_id,step,timestamp,anglez,enmo,awake,seconds_since_start
0,08db4255286f,0,2018-11-05T10:00:00-0400,-30.8453,0.0447,1,0
1,08db4255286f,1,2018-11-05T10:00:05-0400,-34.1818,0.0443,1,5
2,08db4255286f,2,2018-11-05T10:00:10-0400,-33.8771,0.0483,1,10
3,08db4255286f,3,2018-11-05T10:00:15-0400,-34.2821,0.0680,1,15
4,08db4255286f,4,2018-11-05T10:00:20-0400,-34.3858,0.0768,1,20


In [ ]:
# creating possible time covariates for model 
start_time = datetime.strptime(zzz['timestamp'].iloc[0], '%Y-%m-%dT%H:%M:%S%z')
zzz['seconds_since_start'] = zzz['timestamp'].apply(lambda x: (datetime.strptime(x, '%Y-%m-%dT%H:%M:%S%z') - start_time).seconds)
zzz = zzz.drop(columns=['timestamp', 'step'])

In [ ]:
# sorte by series_id & seconds_since_start
zzz_sorted = zzz.sort_values(by=['series_id', 'seconds_since_start'])

# Standardize feats: anglez & enmo 
zzz_sorted['anglez_stand'] = (zzz_sorted['anglez'] - zzz_sorted['anglez'].mean()) / zzz_sorted['anglez'].std()
zzz_sorted['enmo_stand'] = (zzz_sorted['enmo'] - zzz_sorted['enmo'].mean()) / zzz_sorted['enmo'].std()

# #set for Linear Model
feats = [col for col in zzz_sorted.columns if 'stand' in col]
X = zzz_sorted[feats]
y = zzz_sorted['awake']

# split the data (79-21)
train_size = int(0.79 * len(zzz_sorted))
X_train, X_test = X.iloc[:train_size], X.iloc[train_size:]
y_train, y_test = y.iloc[:train_size], y.iloc[train_size:]

In [ ]:
file_path1 = 'C:/temp/X_train.csv'
file_path2 = 'C:/temp/y_train.csv'
file_path3 = 'C:/temp/X_test.csv'
file_path4 = 'C:/temp/y_test.csv'
X_train.to_csv(file_path1, index=False)
y_train.to_csv(file_path2, index=False)
X_test.to_csv(file_path3, index=False)
y_test.to_csv(file_path4, index=False)